In [1]:
from getpass import getpass
import kdbai_client as kdbai

## Connect to KDB.AI cloud instance and verify table exists and has data

In [2]:
KDBAI_ENDPOINT = getpass("KDB.AI endpoint: ")
KDBAI_API_KEY = getpass("KDB.AI API key: ")
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

KDB.AI endpoint:  ········
KDB.AI API key:  ········


In [3]:
table = session.table("openai_pdf")

In [4]:
table.query()

,Embeddings,Sentences
0,"[-0.00248929625377059, 0.006371329538524151, -...","Draft version August 14, 2023\nTypeset using L..."
1,"[0.006372543051838875, 0.002492946805432439, -...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes..."
2,"[0.0013814108679071069, 0.0014908972661942244,...",We define a novel framework: firstly to predic...
3,"[0.014146137051284313, -0.0005565343308262527,...",We predict the spatial and compositional distr...
4,"[0.01576417125761509, 0.004918665625154972, 0....",Selecting ISO water mass\nfraction as an examp...
...,...,...
586,"[-0.00573846697807312, -0.006318436004221439, ...","2021, ApJ, 922, 189,\ndoi: 10.3847/1538-4357/a..."
587,"[0.0037363762967288494, -0.0014329071855172515...","2020,\nNature Methods, 17, 261, doi: 10.1038/s..."
588,"[0.0008434861665591598, -0.01327595580369234, ...","A., Frinchaboy, P. M., et al."
589,"[-0.01480394322425127, 0.0031033621635288, -0....","2013, AJ, 146, 81, doi: 10.1088/0004-6256/146/..."


## Export Vector DB data using vector-io 

In [5]:
%run src/export_vdf.py kdbai

Enter the KDB.AI endpoint instance:  https://cloud.kdb.ai/instance/pwjhnuwjez
Enter the name of table to export: openai_pdf
Enter the embedding model used: text-embedding-ada-002
Enter your KDB.AI API key:  ········


Export to disk completed. Exported to: vdf_20240205_101022_82f0d/
Time taken to export data:  00:00:20


## Drop table from KDB.AI instance and query to confirm its deleted

In [5]:
table.drop()

True

In [7]:
try:
    table.query()
    time.sleep(5)
except kdbai.KDBAIException as e:
    print(f"Caught an exception: {e}")

Caught an exception: Failed to process the query {'table': 'openai_pdf'} on table named: openai_pdf, because of: Table not found, table=openai_pdf.


## Try to import the table data into KDB.AI using vector-io

In [6]:
%run src/import_vdf.py  -d "vdf_20240205_101022_82f0d" kdbai

/Users/christos/anaconda3/envs/threedotten/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Enter the endpoint for KDB.AI Cloud instance:  https://cloud.kdb.ai/instance/pwjhnuwjez
Enter your KDB.AI API key:  ········


Table created
Inserted 250 out of 591 rows.
Inserted 500 out of 591 rows.
Inserted 591 out of 591 rows.
Data fully added
Time taken: 37.84 seconds


## Confirm table exists and data available

In [9]:
table.query()

,Embeddings,Sentences
0,"[-0.00248929625377059, 0.006371329538524151, -...","Draft version August 14, 2023\nTypeset using L..."
1,"[0.006372543051838875, 0.002492946805432439, -...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes..."
2,"[0.0013814108679071069, 0.0014908972661942244,...",We define a novel framework: firstly to predic...
3,"[0.014146137051284313, -0.0005565343308262527,...",We predict the spatial and compositional distr...
4,"[0.01576417125761509, 0.004918665625154972, 0....",Selecting ISO water mass\nfraction as an examp...
...,...,...
586,"[-0.00573846697807312, -0.006318436004221439, ...","2021, ApJ, 922, 189,\ndoi: 10.3847/1538-4357/a..."
587,"[0.0037363762967288494, -0.0014329071855172515...","2020,\nNature Methods, 17, 261, doi: 10.1038/s..."
588,"[0.0008434861665591598, -0.01327595580369234, ...","A., Frinchaboy, P. M., et al."
589,"[-0.01480394322425127, 0.0031033621635288, -0....","2013, AJ, 146, 81, doi: 10.1088/0004-6256/146/..."
